# Procesamiento Digital de Imágenes Satelitales
# Instituto Gulich - Mayo - Julio 2025

---
#  Clase 06c: Análisis exploratorio de datos
---


### Objetivos de la notebook:

- En esta notebook esperamos que aprendas a mirar en conjunto datos raster y ROIS vectoriales.

### Datos con los que trabajaremos en esta Notebook:

- Los recortes de Sentinel-2 del Dique Roggero 
    - **S2_dique_20191120.tif** y
    - **S2_dique_20181006.tif**
- Capa vectorial de ROIS
    - **rois_dique.geojson**

```python
import os
import sys
sys.path.append('../')

import numpy as np
import matplotlib.pyplot as plt

import rasterio
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.mask import mask

import geopandas as gpd

import pandas as pd
import seaborn as sns

from funciones import *
```

```python
path_proc = '../data/proc/'
path_out = '../data/out/'
path_vector = '../data/raw_data/vector/'
```

```python
raster_fn = os.path.join(path_proc, 'S2_dique_20181006.tif')

with rasterio.open(raster_fn) as src:
    img = src.read()
    gt = src.transform

```

```python
rois_vec = gpd.read_file(os.path.join(path_vector, 'rois_dique.geojson'))
rois_vec.head(10)
```

## Uso de _mask_ para extraer valores de un raster a partir de un ROI
### Profundizando un poco en los arrays enmascarados

Como vimos en notebooks anteriores, es posible generar recortes a partir de nuestros polígonos haciendo uso del módulo [rasterio.mask](https://rasterio.readthedocs.io/en/latest/api/rasterio.mask.html). Veamos un ejemplo con nuestros datos:

```python
# Aquí extraemos la geometría del primer ROI
roi0 = rois_vec['geometry'][0]

# Utilicemos el ROI anterior para extraer los valores de la imagen
with rasterio.open(raster_fn) as src:
     out_image, out_transform = mask(src, [roi0], crop=True)

# Grafiquemos nuestro subset
fig, ax = plt.subplots()
show(nequalize(out_image[[2, 1, 0]], p=1), ax=ax, transform=out_transform)
# rois_vec.loc[[0]].boundary.plot(ax=ax, color='orangered')
# ax.ticklabel_format(style='plain')
plt.show()
```

### Haciendo que mask devuelva un array enmascarado con `filled=false`

```python
with rasterio.open(raster_fn) as src:
    out_image, out_transform = mask(src, [roi0], crop=True, filled=False)
```

```python
type(out_image)
```

```python
out_image[:, 0, 0]
```

```python
out_image.data.shape
```

```python
mascara = out_image.mask[0]

mascara
```

```python
out_image.data[:, ~mascara]
```

## Etiquetado de datos

Para entender mejor nuestros datos, es necesario asociar la información de nuestras imágenes a su respectiva etiqueta, la cual puede provenir por ejemplo de información relevada en campo o interpretación visual sobre las mismas. Para realizar esto, vamos a construir un dataframe que contendrá como _features_ o características la información espectral y, como _label_ o etiqueta el valor correspondiente a cada ROI. Iniciemos con una sola clase:

```python
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt

# archivos
vector_fn = os.path.join(path_vector, 'rois_dique.geojson')
raster_fn = os.path.join(path_proc, 'S2_dique_20181006.tif')

# Cargamos nuestros polígonos
rois_vec = gpd.read_file(vector_fn)

# Definimos un valor fuera de nuestro rango válido para los datos no válidos
nodata = -255
```


Leamos nuestro vector y recortemos a un polígono

```python
# Cargamos nuestros polígonos y nos quedamos con uno de los elementos
clase, geom = rois_vec.loc[0, ['clase', 'geometry']].to_list()

# Cargamos la imagen y la recortamos a nuestro polígono
with rasterio.open(raster_fn) as src:
    clip, _transform = mask(src, [geom], crop=True, nodata=nodata)
```

```python
d, x, y = clip.shape
D = clip.reshape([d, x*y]).T  # rompemos la estructura espacial, pero no espectral
DX = D[np.all(D != nodata, axis=1)]  # Nos quedamos con los valores útiles
DY = np.repeat(clase, DX.shape[0])  # Generamos un array con la etiqueta de la clase
```

Veamos algunos datos etiquetados:

```python
print('Clase  Blue   Green  Red    NIR     NDVI')
for i in range(10):
    j = np.random.randint(DX.shape[0])
    print(f'{DY[j]:6s} {DX[j,0]:.3f}  {DX[j,1]:.3f}  {DX[j,2]:.3f}  {DX[j,3]:.3f}  {DX[j,4]: .3f}')
```

Vamos a extender lo anterior a todas las clases, para lo que asumiremos que ya cargamos nuestros ROIs y definimos nuestro valor _nodata_. En este caso tendremos previmente que generar un array para ir concatenando los datos contenidos en cada ROI, así como sus valores de clase. Además, vamos a generar un listado con las etiquetas posibles:

```python
# primero defino cantidad de atributos = cantidad de bandas en el raster
with rasterio.open(raster_fn) as src:
    d = src.count 

# Clases de los ROIs
clases = list(set(rois_vec['clase']))
# clases = list(rois_vec['clase'].unique())
clases.sort()

# Generamos valores categóricos para nuestras clases
clase_dict = {clase: i for i, clase in enumerate(clases)}
```

Creamos los vectores para almacenar nuestra información de datos y etiquetas

```python
X = np.empty((0, d), dtype=np.float32)  # array vacío para almacenar valores espectrales
Y = np.empty(0, dtype=int)              # array vacío para almacenar etiquetas

```

Con el raster en memoria, iteramos sobre los polígonos y extraemos para cada uno la información que nos interesa, cada vez almacenamos en los array anteriores la información extraida con su respectiva etiqueta:

```python
with rasterio.open(raster_fn) as src:
    for index, row in rois_vec.iterrows():
        clase, geom = row[['clase', 'geometry']].to_list()
        clip, _transform = mask(src, [geom], crop=True, nodata=nodata)
        d, x, y = clip.shape
        D = clip.reshape([d, x*y]).T  # Rompemos la estructura espacial, pero no espectral
        DX = D[np.all(D != nodata, axis=1)]  # Nos quedamos con los valores útiles
        DY = np.repeat(clase, DX.shape[0])  # Generamos un array con la etiqueta de la clase
        X = np.concatenate((X, DX))  # Concatenamos nuestros datos espectrales
        Y = np.concatenate((Y, DY))  # Concatenamos las etiquetas de la clase
```

El array X tiene una fila por cada pixel (con sus datos espectrales), mientras que Y tiene un elemento por cada pixel indicando la clase correspondiente. Veamos algunos datos etiquetados:

```python
R = 10  # cantindad a elegir al azar
N = len(Y)

print('Clase      Blue   Green  Red    NIR     NDVI')
for i in range(R):
    j = np.random.randint(N)
    print(f'{clase_dict[Y[j]]:d}.{Y[j]:<7}: {X[j,0]:.3f}  {X[j,1]:.3f}  {X[j,2]:.3f}  {X[j,3]:.3f}  {X[j,4]: .3f}')
```

## Firmas espectrales

```python
# Visualización
        
# Creamos la figura
B_G_R_NIR = [0.49, 0.56, 0.665, 0.842]
fig, ax = plt.subplots(figsize=(15, 10))

for j, l in enumerate(clases):
    S = np.where(Y==l)[0]
    data_vis_clase = X[S,:-1]
    media_clase = data_vis_clase.mean(axis=0)
    std_clase = data_vis_clase.std(axis=0)
    ax.plot(B_G_R_NIR, media_clase, label=l)
    ax.fill_between(B_G_R_NIR, media_clase - std_clase, media_clase + std_clase, color='b', alpha=.1)

ax.set_ylim(0, 0.4)
ax.legend()
ax.set_xlabel("longitud de onda (nm)")
ax.set_ylabel("reflectancia")

ax.set_title('Firmas espectrales de las clases')
font = {'family':'serif', 'color':'darkgray', 'size':12}

for x, t in zip(B_G_R_NIR, ['Blue', 'Green', 'Red', 'NIR']):
    ax.text(x - 0.005, 0.03, t, fontdict=font)

plt.show()
```

## Boxplots, Violinplots y Scatterplots

```python
# Armo un dataframe con los datos etiquetados
data = pd.DataFrame({'B':X[:,0],
                     'G':X[:,1],
                     'R':X[:,2],
                     'NIR':X[:,3],
                     'NDVI':X[:,4],
                     'clase':Y})

data['clase_cat'] = data['clase'].map(clase_dict).astype("category")

# y los grafico de diferentes formas
fig, ax = plt.subplots(figsize=(15, 10))
sns.boxplot(data=data, x='clase', y='NDVI', ax=ax)
plt.show()
```

```python
fig, ax = plt.subplots(figsize=(15, 10))
sns.violinplot(data=data, x='clase', y='NDVI', ax=ax)
plt.show()
```

```python
fig, ax = plt.subplots(figsize=(10, 5))
sns.scatterplot(data=data, x='R', y='G',hue='clase', alpha=0.3, s=15, ax=ax)
plt.show()
```

### 9.1 Ejercicio de visualización de datos (opcional)
1. Repetir el violinplot pero graficando solo para la clase _rural_
1. Repetir el scatterplot usando los ejes:
    1. Red y NIR
    1. NDVI y Blue
    1. NDVI y Green

### 9.2 Ejercicio de etiquetado
1. En los diferentes gráficos se observa que la clase _rural_ está compuesta de al menos dos clases mezcaldas. 
    1. Mirando tu último violin-plot, definir un umbral que separe las dos potenciales clases diferentes.
    1. Generar un nuevo geoDataFrame llamado rois_rural (a partir de rois_vec) que tenga solo aquellos ROIS de la clase 'rural'. Luego agregar una columna con el promedio de los valores de NDVI del ROI correspondiente.
    1. Modificar la columna 'clase' de rois_rural de manera de distinguir los dos tipos de ROIS rurales (los que tengan NDVI alto y bajo, de acuerdo al umbral seleccionado).
    1. Graficar la imagen Sentinel-2 RGB ecualizada y sobre ella los bordes de los dos tipos de ROIS rurales con colores diferentes.
    1. Observando estas últimas dos imágenes tratar de entender a qué corresponden las dos subclases de 'rural'. Asignarles un nombre adecuado.
    1. Modificar el archivo _geoJSON_ usando estas nuevas etiquetas, según corresponda, para la vieja clase 'rural'.
    1. Repetir los gráficos anteriores (firmas espectrales, violinplots, y scatterplots) considerando las nuevas etiquetas.


```python
## Ayuda Ejercicio 9.2 B

# Generar un nuevo GeoDataFrame llamados rois_rural a partir de rois_vec
rois_rural = data.loc[data['clase']=='rural', :].copy()

# Agregar una columna con el promedio de los valores de NDVI del ROI correspondiente:
with rasterio.open(raster_fn) as src:
    for index, row in rois_rural.iterrows():
        geom = ...
        clip, _transform = mask(..., ..., crop=..., nodata=...)
        d, x, y = clip.shape
        D = clip.reshape([d, ...]).T  # Rompemos la estructura espacial, pero no espectral
        DX = D[np.all(D != nodata, axis=1)]  # Nos quedamos con los valores útiles
        mndvi = DX[:,-1].mean()  # Calculamos el NDVI medio
        rois_rural.loc[..., 'NDVI_medio'] = mndvi  # Asignamos al polígono el valor de NDVI medio
```